# 🎬 CINDERGRACE - ComfyUI Cloud Backend

**Reihenfolge:**
1. ⚙️ Konfiguration
2. 💾 Google Drive verbinden
3. 📦 ComfyUI installieren
4. 🔑 HuggingFace Login (für geschützte Modelle)
5. 📥 Modelle herunterladen
6. 🚀 ComfyUI starten

---

## 1️⃣ Konfiguration

In [ ]:
#@title ⚙️ Einstellungen

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = False  #@param {type:"boolean"}
INSTALL_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_NODE_DEPENDENCIES = True  #@param {type:"boolean"}
INSTALL_FLUXTRAINER = False  #@param {type:"boolean"}

if USE_GOOGLE_DRIVE:
    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
else:
    WORKSPACE = "/content/ComfyUI"

# Pfade für Modelle (ComfyUI Struktur)
DIFFUSION_MODELS_DIR = f"{WORKSPACE}/models/diffusion_models"
TEXT_ENCODERS_DIR = f"{WORKSPACE}/models/text_encoders"
VAE_DIR = f"{WORKSPACE}/models/vae"
LORAS_DIR = f"{WORKSPACE}/models/loras"
CHECKPOINTS_DIR = f"{WORKSPACE}/models/checkpoints"

print(f"📁 Workspace: {WORKSPACE}")
print(f"🔄 Update ComfyUI: {UPDATE_COMFY_UI}")
print(f"🧩 ComfyUI-Manager: {INSTALL_COMFYUI_MANAGER}")
print(f"🎓 FluxTrainer: {INSTALL_FLUXTRAINER}")
print("✅ Konfiguration geladen")

## 2️⃣ Google Drive verbinden

In [ ]:
#@title 💾 Google Drive mounten

if USE_GOOGLE_DRIVE:
    print("🔗 Verbinde mit Google Drive...")
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive verbunden")
else:
    print("⏭️ Google Drive übersprungen (Daten gehen bei Session-Ende verloren!)")

## 3️⃣ ComfyUI installieren

In [ ]:
#@title 📦 ComfyUI + Manager + Dependencies

import os
import time

start_time = time.time()

# ComfyUI klonen oder updaten
if not os.path.exists(WORKSPACE):
    print("="*50)
    print("📥 ComfyUI wird heruntergeladen (Erstinstallation)...")
    print("   Das kann 5-10 Minuten dauern.")
    print("="*50)
    !git clone --progress https://github.com/comfyanonymous/ComfyUI {WORKSPACE}
    print(f"✅ ComfyUI geklont ({time.time() - start_time:.0f}s)")
elif UPDATE_COMFY_UI:
    print("🔄 ComfyUI wird aktualisiert...")
    %cd {WORKSPACE}
    !git pull
    print(f"✅ ComfyUI aktualisiert ({time.time() - start_time:.0f}s)")
else:
    print("⏭️ ComfyUI Update übersprungen (bereits installiert)")

%cd {WORKSPACE}

# Dependencies
print("\n" + "="*50)
print("📦 Installiere Python Dependencies...")
print("   Das kann 2-3 Minuten dauern.")
print("="*50)
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121
print(f"✅ Dependencies installiert ({time.time() - start_time:.0f}s)")

# ComfyUI-Manager
if INSTALL_COMFYUI_MANAGER:
    print("\n🧩 ComfyUI-Manager...")
    manager_path = f"{WORKSPACE}/custom_nodes/ComfyUI-Manager"
    if not os.path.exists(manager_path):
        print("   Klone ComfyUI-Manager...")
        !git clone https://github.com/ltdrdata/ComfyUI-Manager {manager_path}
    else:
        print("   Manager bereits vorhanden, update...")
        %cd {manager_path}
        !git pull
    !chmod 755 {manager_path}/*.sh 2>/dev/null || true
    print("✅ ComfyUI-Manager bereit")

# FluxTrainer (optional)
if INSTALL_FLUXTRAINER:
    print("\n🎓 FluxTrainer (Cindergrace Fork)...")
    trainer_path = f"{WORKSPACE}/custom_nodes/ComfyUI-FluxTrainer"
    if os.path.exists(trainer_path):
        print("   Entferne alten FluxTrainer...")
        !rm -rf {trainer_path}
    print("   Klone Cindergrace Fork...")
    !git clone https://github.com/goettemar/ComfyUI-FluxTrainer-Cindergrace.git {trainer_path}
    print("   Installiere FluxTrainer Dependencies...")
    !pip install -r {trainer_path}/requirements.txt
    print("✅ FluxTrainer bereit")

# Node Dependencies
%cd {WORKSPACE}
if INSTALL_NODE_DEPENDENCIES and INSTALL_COMFYUI_MANAGER:
    print("\n📦 Custom Node Dependencies...")
    !pip install GitPython
    !python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies
    print("✅ Node Dependencies installiert")

# Modell-Ordner erstellen
print("\n📁 Erstelle Modell-Ordner...")
for d in [DIFFUSION_MODELS_DIR, TEXT_ENCODERS_DIR, VAE_DIR, LORAS_DIR, CHECKPOINTS_DIR]:
    os.makedirs(d, exist_ok=True)
    print(f"   {d.split('/')[-1]}/")

total_time = time.time() - start_time
print("\n" + "="*50)
print(f"✅ ComfyUI bereit! (Gesamtzeit: {total_time/60:.1f} Minuten)")
print("="*50)

---
## 4️⃣ HuggingFace Login

**Für geschützte Modelle (Flux Krea-Dev, VAE) benötigst du einen HuggingFace Account:**

1. Account erstellen: https://huggingface.co/join
2. Token erstellen: https://huggingface.co/settings/tokens
3. Token unten einfügen

In [ ]:
#@title 🔑 HuggingFace Token eingeben

#@markdown Erstelle einen Token hier: https://huggingface.co/settings/tokens
#@markdown (Read-Berechtigung reicht)

HF_TOKEN = ""  #@param {type:"string"}

print("📦 Installiere huggingface_hub...")
!pip install huggingface_hub

if HF_TOKEN:
    from huggingface_hub import login
    login(token=HF_TOKEN)
    print("✅ Bei HuggingFace angemeldet")
else:
    print("⚠️ Kein Token eingegeben - nur öffentliche Modelle verfügbar")
    print("   Für Flux Krea-Dev und VAE wird ein Token benötigt!")

---
## 5️⃣ Modelle herunterladen

**Cindergrace Keyframe Workflow benötigt:**
- Flux Krea-Dev (diffusion_models) - 23GB, Auth erforderlich
- Flux VAE (vae) - Auth erforderlich
- T5 XXL (text_encoders)
- CLIP L (text_encoders)

In [ ]:
#@title 📥 Cindergrace Keyframe Modelle (flux_keyframe_hq)

#@markdown ### Diffusion Model (benötigt HF Auth)
DOWNLOAD_FLUX_KREA_DEV = False  #@param {type:"boolean"}

#@markdown ### VAE (benötigt HF Auth)
DOWNLOAD_FLUX_VAE = False  #@param {type:"boolean"}

#@markdown ### Text Encoders
DOWNLOAD_T5XXL = False  #@param {type:"boolean"}
DOWNLOAD_CLIP_L = False  #@param {type:"boolean"}

import os
import time

start_time = time.time()

if DOWNLOAD_FLUX_KREA_DEV:
    print("="*50)
    print("📥 Flux Krea-Dev (~23GB) → diffusion_models/")
    print("   Das dauert ca. 10-15 Minuten...")
    print("="*50)
    output = f"{DIFFUSION_MODELS_DIR}/flux1-krea-dev.safetensors"
    if not os.path.exists(output):
        !huggingface-cli download black-forest-labs/FLUX.1-Krea-dev flux1-krea-dev.safetensors --local-dir {DIFFUSION_MODELS_DIR}
        print(f"✅ Flux Krea-Dev heruntergeladen ({time.time() - start_time:.0f}s)")
    else:
        print("⏭️ Bereits vorhanden, übersprungen")

if DOWNLOAD_FLUX_VAE:
    print("\n📥 Flux VAE → vae/")
    output = f"{VAE_DIR}/ae.safetensors"
    if not os.path.exists(output):
        !huggingface-cli download black-forest-labs/FLUX.1-dev ae.safetensors --local-dir {VAE_DIR}
        print("✅ VAE heruntergeladen")
    else:
        print("⏭️ Bereits vorhanden")

if DOWNLOAD_T5XXL:
    print("\n📥 T5 XXL FP16 (~9GB) → text_encoders/")
    output = f"{TEXT_ENCODERS_DIR}/t5xxl_fp16.safetensors"
    if not os.path.exists(output):
        !wget --progress=bar:force -O "{output}" "https://huggingface.co/Comfy-Org/stable-diffusion-3.5-fp8/resolve/main/text_encoders/t5xxl_fp16.safetensors"
        print("✅ T5 XXL heruntergeladen")
    else:
        print("⏭️ Bereits vorhanden")

if DOWNLOAD_CLIP_L:
    print("\n📥 CLIP L → text_encoders/")
    output = f"{TEXT_ENCODERS_DIR}/clip_l.safetensors"
    if not os.path.exists(output):
        !wget --progress=bar:force -O "{output}" "https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors"
        print("✅ CLIP L heruntergeladen")
    else:
        print("⏭️ Bereits vorhanden")

print("\n" + "="*50)
print(f"✅ Keyframe Modelle abgeschlossen ({time.time() - start_time:.0f}s)")
print("="*50)

In [ ]:
#@title 📥 Wan 2.2 Video Modelle (für Video Generator)

DOWNLOAD_WAN_14B = False  #@param {type:"boolean"}
DOWNLOAD_WAN_5B = False  #@param {type:"boolean"}

if DOWNLOAD_WAN_14B:
    print("📥 Wan 2.2 14B (~28GB)...")
    # TODO: Wan 2.2 HuggingFace Link
    print("⚠️ URL noch nicht hinterlegt - bitte manuell hinzufügen")

if DOWNLOAD_WAN_5B:
    print("📥 Wan 2.2 5B (~10GB)...")
    # TODO: Wan 2.2 5B HuggingFace Link
    print("⚠️ URL noch nicht hinterlegt - bitte manuell hinzufügen")

print("✅ Video Modelle abgeschlossen")

In [ ]:
#@title 📥 Eigene URL herunterladen

#@markdown Lade ein Modell von einer beliebigen URL herunter:
DOWNLOAD_URL = ""  #@param {type:"string"}
FILENAME = "model.safetensors"  #@param {type:"string"}

#@markdown Zielordner:
TARGET_FOLDER = "diffusion_models"  #@param ["diffusion_models", "text_encoders", "vae", "loras", "checkpoints"]

if DOWNLOAD_URL:
    target_dirs = {
        "diffusion_models": DIFFUSION_MODELS_DIR,
        "text_encoders": TEXT_ENCODERS_DIR,
        "vae": VAE_DIR,
        "loras": LORAS_DIR,
        "checkpoints": CHECKPOINTS_DIR
    }
    target = target_dirs[TARGET_FOLDER]
    output_path = f"{target}/{FILENAME}"
    
    print(f"📥 Lade herunter: {DOWNLOAD_URL[:50]}...")
    print(f"📁 Ziel: {output_path}")
    print("="*50)
    
    !wget --progress=bar:force -c -O "{output_path}" "{DOWNLOAD_URL}"
    
    print(f"\n✅ Download abgeschlossen: {FILENAME}")
else:
    print("ℹ️ Keine URL angegeben - übersprungen")

In [ ]:
#@title 📋 Vorhandene Modelle anzeigen

import os

def list_models(path, name):
    if os.path.exists(path):
        files = [f for f in os.listdir(path) if f.endswith(('.safetensors', '.ckpt', '.pt', '.bin', '.gguf'))]
        if files:
            print(f"\n📁 {name}:")
            for f in sorted(files):
                size = os.path.getsize(os.path.join(path, f)) / (1024**3)
                print(f"   ✓ {f} ({size:.1f} GB)")
        else:
            print(f"\n📁 {name}: (leer)")
    else:
        print(f"\n📁 {name}: (Ordner existiert nicht)")

print("="*50)
print("📦 Modelle in Google Drive:")
print("="*50)

list_models(DIFFUSION_MODELS_DIR, "Diffusion Models")
list_models(TEXT_ENCODERS_DIR, "Text Encoders")
list_models(VAE_DIR, "VAE")
list_models(LORAS_DIR, "LoRAs")
list_models(CHECKPOINTS_DIR, "Checkpoints")

---
## 6️⃣ ComfyUI starten

**Nach dem Start:** Die Cloudflare-URL kopieren und in CINDERGRACE Settings einfügen.

In [ ]:
#@title 🚀 ComfyUI mit Cloudflare Tunnel starten

print("📦 Installiere Cloudflare Tunnel...")
!wget -P ~ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket

def start_tunnel(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        sock.close()
        if result == 0:
            break
    
    print("\n" + "="*60)
    print("🎬 CINDERGRACE Backend bereit!")
    print("="*60)
    
    p = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    
    for line in p.stderr:
        l = line.decode()
        if "trycloudflare.com" in l:
            url = l[l.find("http"):].strip()
            print(f"\n📋 Diese URL in CINDERGRACE Settings eintragen:")
            print(f"\n   {url}")
            print(f"\n💡 Settings → Backend hinzufügen → URL einfügen")
            print("="*60 + "\n")

print("\n🚀 Starte ComfyUI...")
threading.Thread(target=start_tunnel, daemon=True, args=(8188,)).start()

%cd {WORKSPACE}
!python main.py